In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import sys
import os
import re
PATH = os.path.join(".", "4PiZernikes")
FILES = [{'filename':fn,
         'offset': float(re.match(r".*([\+|-][0-9]+\.[0-9]+).*", fn).groups()[0]),}
         for fn in os.listdir(PATH) if re.match(r"zernike", fn)]

for f in FILES:
    with open(os.path.join(PATH, f['filename']), 'r', encoding='utf-16-le') as fh:
        row = ' '
        # Skip header
        while(row and not re.match(r"\s*Field:", row)):
            row = fh.readline()
        raw = list(fh)
        data = []
        for row in raw:
            data.append([float(col) for col in re.split(r"\s*", row.strip())])
        f['field'] = np.array([d[0] for d in data])
        f['zernike'] = np.array([d[1:] for d in data])
        
with open(os.path.join(PATH, "mag.TXT"), encoding='utf-16-le') as fh:
    row = ' '
    while(row and not re.match("\s*X\s*Y", row)):
        row = fh.readline()
    raw = list(fh)
    MAG = np.split(np.array([[float(col) for col in re.split(r"\s*", row.strip())] 
                                    for row in raw]).T, 2)[1].flatten()

FIELD = FILES[0]['field']
order = np.argsort([F['offset'] for F in FILES])
OFFSET = np.array([FILES[o]['offset'] for o in order])
ALLDATA = np.array([FILES[o]['zernike'] for o in order])

Using matplotlib backend: TkAgg


C:\python3\python-3.5.2.amd64\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [2]:
# Surface plot for one mode.
mode = 0
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(OFFSET, FIELD)
ax.plot_surface(X,Y, ALLDATA[:,:,mode].T, rstride=1, cstride=1)
fig.show()

In [4]:
# Interactive plot for all modes.
import numpy as np
import matplotlib.pyplot as plt

def do_interp(offset, data):
    for z in range(data.shape[1]):
        for f in range(data.shape[0]):
            data[f,z] = np.interp([offset], OFFSET, ALLDATA[:,f,z])
    mag = np.interp([offset], OFFSET, MAG)
    return data, mag

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.1, bottom=0.25, right=0.8)
axoffset = plt.axes([0.1, 0.1, 0.7, 0.03])

data, mag = do_interp(0, np.zeros_like(FILES[0]['zernike']))
lines = ax.plot(FIELD, data)
for (i, l) in enumerate(lines):
    l.set_linestyle(['-','-.',':','--'][int(i/7)])

ax.set_title("Mag = %.2f" % mag)
ax.set_xlim(FIELD.min(), FIELD.max())
ax.set_ylim(ALLDATA.min(), ALLDATA.max())
ax.set_xlabel('field / mm')
ax.set_ylabel('Zernike coefficient / waves')
ax.legend([l for l in lines], range(len(lines)), loc='center left', 
          bbox_to_anchor=(1.05, 0.5), 
          bbox_transform=ax.transAxes, 
          title="Standard\nZernike\ncoefficient")


soffset = Slider(axoffset, '', OFFSET.min(), OFFSET.max(), valinit=0, closedmin=True, closedmax=True)
axoffset.set_xlabel('offset / mm')


def update(val):
    offset = soffset.val
    soffset.poly.set_color(['red','blue'][offset >= soffset.valinit])
    data, mag = do_interp(offset, np.zeros_like(FILES[0]['zernike']))
    ax.set_title("Mag = %.2f" % mag)
    for i, l in enumerate(lines):
        l.set_ydata(data[:,i])
    #ax.relim()
    #ax.autoscale(enable=True, axis='y')
    #ax.autoscale(False, True)
    
soffset.on_changed(update)

plt.show()